# Deep learning from scratch

# Learning objectives of the notebook

- Get used to working with *PyTorch Tensors*, the core data structure needed for working with neural networks;
- Practice using the `autograd` capabilities of PyTorch Tensors to carry out backpropagation without all the pain;
- Apply the useful PyTorch `torch.no_grad` context manager for managing memory consumption;
- Convert a NumPy-based gradient descent algorithm into one relying on PyTorch Tensors!

# PyTorch Basics

[PyTorch](http://pytorch.org) is a Python-based scientific computing package to support deep learning research. It provides tensor support (a replacement of NumPy, of sorts) to provide a fast & flexible platform for experimenting with neural networks.

In [1]:
import torch
import numpy as np
print(f'PyTorch version: {torch.__version__}')
print(f'NumPy version:   {np.__version__}')

PyTorch version: 1.4.0
NumPy version:   1.19.1


The principal data structures in PyTorch are *tensors*; these are pretty much the same as standard multidimensional NumPy arrays. To illustrate this, let's construct a matrix of zeros (of `long` or 64 bit integer `dtype`) in NumPy, and then in PyTorch.

In [2]:
# zeros construction in NumPy
x_np = np.zeros((2,4), dtype=np.int64)
print(x_np, x_np.dtype)

[[0 0 0 0]
 [0 0 0 0]] int64


In [3]:
# zeros construction in PyTorch
x = torch.zeros(2, 4, dtype=torch.long) # Observe difference in calling syntax!
print(x, x.dtype)

tensor([[0, 0, 0, 0],
        [0, 0, 0, 0]]) torch.int64


You can query a tensor's size (dimensions) with the `size` method (contrast with NumPy array `shape` attribute).

In [4]:
print(x)
print(x.size())   # "size" is *method* for torch tensors
print(x_np.shape) # 'shape' is *attribute* returning tuple
print(x_np.size)  # "size"  is *attribute* for np arrays

tensor([[0, 0, 0, 0],
        [0, 0, 0, 0]])
torch.Size([2, 4])
(2, 4)
8


In [5]:
# torch.Tensor.size() yields subclass of Python tuple
print(type(x.size()))

<class 'torch.Size'>


As with NumPy, there are a variety of PyTorch data types for arrays:

|  NumPy dtype | PyTorch dtype | Alternative | Tensor class |
|:-:|:-:|:-:|:-:|
| `np.int16`  |`torch.Int16`  |`torch.short` |`ShortTensor` |
| `np.int32`  |`torch.Int32`  |`torch.Int`   |`IntTensor`   |
| `np.int64`  |`torch.Int64`  |`torch.long`  |`LongTensor`  |
| `np.float16`|`torch.float16`|`torch.half`  |`HalfTensor`  |
| `np.float32`|`torch.float32`|`torch.float` |`FloatTensor` |
| `np.float64`|`torch.float64`|`torch.double`|`DoubleTensor`|


Many functions and methods in PyTorch have similar names to NumPy functions & methods:

In [6]:
print(torch.empty(3, 4, dtype=torch.short), end='\n\n')  # like numpy.empty
print(torch.ones(3, 4, dtype=torch.short), end='\n\n')   # like numpy.ones
print(torch.randn(3, 4, dtype=torch.float), end='\n\n')  # like numpy.random.randn

tensor([[    0,     0,     0,     0],
        [26169, 12595, 12601, 13881],
        [25141, 12852, 24884, 14645]], dtype=torch.int16)

tensor([[1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1]], dtype=torch.int16)

tensor([[-0.8549,  1.1673, -1.0949,  2.3805],
        [-0.3755,  0.4157,  0.5453, -0.5259],
        [ 2.4635,  0.2841,  0.2915,  1.4225]])



You can also construct PyTorch tensors from lists of numerical data or NumPy arrays.

In [7]:
# Constructing tensors from lists of data
print(torch.tensor([1,2,3]).dtype)  # inferred to be 64 bit integers
print(torch.Tensor([1,2,3]).dtype)  # specifically cast to 32 bit floats

torch.int64
torch.float32


Notice the factory function `torch.tensor` differs from the class constructor `torch.Tensor`. The former *infers* the data type of the tensor to construct from the numerical data input. By constrast, the latter is  just an alias for `Torch.FloatTensor` (i.e., the data are cast to 32 bit floating point numbers).

PyTorch Tensors can be converted to NumPy arrays using the method `torch.Tensor.numpy`:

In [8]:
a = torch.rand(2,3)   # first, construct a random PyTorch tensor
print(a)
print(a.dtype)

tensor([[0.4541, 0.7676, 0.9434],
        [0.9712, 0.7082, 0.9782]])
torch.float32


In [9]:
b = a.numpy()        # converts to NumPy array (shallow copy; use .copy() for deep copy)
print(b)
print(type(b))

[[0.45411032 0.76763356 0.94335616]
 [0.97120625 0.7082082  0.9781515 ]]
<class 'numpy.ndarray'>


In [10]:
b[0,0] = 6

[What is PyTorch?](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html)  at [`pytorch.org`](https://pytorch.org) provides a quick tour through related topics (e.g., tensor indexing, arithmetic operations, elementwise functions, linear algebra, etc.). For the most part, these resemble (although not perfectly) the same corresponding tasks in NumPy.

# Backpropagation with `autograd`

Why PyTorch Tensors when all they seem to offer is the same functionality of NumPy arrays? Another related question is why go through the trouble to reimplement everything that's done in NumPy in PyTorch (with slightly different names & APIs)? There are two principle advantages that systems like PyTorch have over NumPy for numerical computing:

1. **Automatic differentiation**: PyTorch includes a package called [`autograd`](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html) that computes the backpropagation algorithm for users. As such, the management of gradients (and the associated memory needed) is significantly simplified with the PyTorch framework. This is, of course, very important for implementing gradient descent.
2. **GPU computation**: GPUs (graphical processing units) are widely available to speed up computation. However, GPU programming remains challenging for most developers with the memory management issues associated with moving data onto GPUs to speed up computation. With PyTorch, much of the work of moving tensors onto GPUs is handled for the user which makes programing with GPUs much easier... and this in turn speeds up a lot of neural network training.

If we examine the object `a` created above, you can see it has an attribute `device` that can be set in various ways depending on the availability of GPU hardware.

In [13]:
a.device # PyTorch tensors have a `device` attribute
# Common alternatives: device(type='cpu'), device(type='cuda'), etc.

device(type='cpu')

We'll focus mostly on automatic differentiation today as supported by the [`autograd`](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html) module. Remember, our main reason for wanting to do this is to compute gradients as needed to train neural network parameters (weights & biases) with gradient descent. In PyTorch, automatic differentiation of tensors is achieved using through setting the `requires_grad` attribute to `True` for all relevant `torch.Tensor`s on construction (the default value is `False`). Alternatively, there is also a method `.requires_grad_( ... )` that modifies the `requires_grad` flag in-place (default value `False`).

Once tensors are defined with the `requires_grad` attribute set correctly, additional space is allocated for intermediate computations (remember all the extra lists of arrays we had to maintain explicitly within the `forward` and `backward` functions?). These are used when calling `torch.Tensor.backward()` to compute all gradients recursively. The intermediate gradients computed can then be retrieved using the attribute `torch.Tensor.grad`.

### Backpropagation example

Let's consider a simple polynomial function like below applied to a scalar value $x$:

$\begin{aligned} &\mathrm{Function:} & f(x) &= 3x^4 -2x^3 + 4x^2 - x + 5 \\
&\mathrm{Derivative:} & f'(x) &= 12x^3 -6 x^2 + 8x -1\end{aligned}$

1. Create tensor `x` with the attribute `requires_grad=True` set in the constructor.

In [14]:
x = torch.tensor(2.0, requires_grad=True)

2. Map the polynomial function $f$ onto tensor the `x` and assign the result to `y`. You can verify explicitly that, when $x=2$, $f(x)=51$:
 $$f(2)=3(2)^4 - 2(2)^3 + 4(2)^2 -(2) +5 = 48-16+16-2+5 = 51$$

In [17]:
y = 3*x**4 - 2*x**3 + 4*x**2 - x + 5  # Write out computation of y explicitly.

print(y) # Notice y has a new attribute: grad_fn
print(y.grad_fn)

tensor(51., grad_fn=<AddBackward0>)


The object `y` has an associated gradient function accessible as `y.grad_fn`. When `y` is computed and stored, a set of algebraic operations is applied to the tensor `x`. If the derivatives of those operations are known, the `autograd` package provides support for computing those derivatives (that's what the `AddBackward0` object is). Invoking `y.backward()`, then, computes the value of *gradient* of `y` with respect to `x` evaluated at `x==2`:

$$f'(2) = 12(2^3) - 6(2^2) + 8(2) - 1 = 96-24+16-1 = 87. $$

Notice that the computed gradient value is stored in the attribute `x.grad` of the original tensor `x`.

In [18]:
y.backward() # Compute derivatives and propagate values back through tensors on which y depends

print(x.grad)  # Expect the value 87 as a singleton tensor

tensor(87.)


Notice that invoking `y.backward()` a second time raises an exception. This is because the intermediate arrays required to execute the backpropagation have been released (i.e., the memory has been deallocated).

In [ ]:
y.backward() # Yields a RuntimeError

### Another backpropagation example

+ Use $z = \cos(u)$ with $u=x^2$ at $x=\sqrt{\frac{\pi}{3}}$
+ Expect $z=\frac{1}{2}$ when $x=\sqrt{\frac{\pi}{3}}$

In [20]:
x = torch.tensor([np.sqrt(np.pi/3)], requires_grad=True)
u = x ** 2
z = torch.cos(u)
print(f'x: {x}\nu: {u}\nz: {z}')

x: tensor([1.0233], requires_grad=True)
u: tensor([1.0472], grad_fn=<PowBackward0>)
z: tensor([0.5000], grad_fn=<CosBackward>)


+ Expect 
  $$\frac{dz}{dx} = \frac{dz}{du} \frac{du}{dx} = (-\sin u) (2 x) = \sqrt{\pi}$$
  when $x=\sqrt{\frac{\pi}{3}}$

In [21]:
# Now apply backward for backpropagation of derivate values
z.backward()

In [22]:
print(f'x.grad:\t\t\t\t\t\t{x.grad}')
x, u = x.item(), u.item() # extract scalar values
print(f'Computed derivative using analytic formula:\t{-np.sin(u)*2*x}')

x.grad:						tensor([-1.7725])
Computed derivative using analytic formula:	-1.7724540834705453


Notice that the tensors `x`, `u`, and `z` are all singleton tensors. The method `item` is used to extract a scalar entry out of a singleton tensor.

# Building a Neural Network in PyTorch

Let's now use an approach adapted from one by [Justin Johnson](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html)  (BSD Clause-3 License). The goal is to convert a NumPy-constructed gradient descent process modelling a feed-forward neural network into a PyTorch neural network. The architecture is similar to the one constructed in the last notebook.

+ The input vectors are assumed to have $784(=28^2)$ features.
+ The first layer is a hidden layer with 100 units and a *rectified linear unit* activation function (often called $\mathrm{ReLU}$):

$$ \mathrm{ReLU}(x) = \begin{cases} x, & \mathrm{if\ }x>0 \\ 0 & \mathrm{otherwise} \end{cases} \quad\Rightarrow\quad
\mathrm{ReLU}'(x) = \begin{cases} 1, & \mathrm{if\ }x>0 \\ 0 & \mathrm{otherwise} \end{cases}.
$$

+ The final output layer has 10 units and the activation function associated with this layer is the identity map.

The loop provided below does not use functions to represent the initialization, forward propagation, backpropagation, and update steps of the steepest descent process. You'll use this as a starting point to develop a PyTorch version of this gradient descent loop.

In [23]:
N_batch, dimensions = 64, [784, 100, 10]

# Create random input and output data
X = np.random.randn(dimensions[0], N_batch)
y = np.random.randn(dimensions[-1], N_batch)

# Randomly initialize weights & biases
W1 = np.random.randn(dimensions[1], dimensions[0])
W2 = np.random.randn(dimensions[2], dimensions[1])
b1 = np.random.randn(dimensions[1], 1)
b2 = np.random.randn(dimensions[2], 1)

eta, MAXITER, SKIP = 5e-6, 2500, 100
for epoch in range(MAXITER):
    # Forward propagation: compute predicted y
    Z1 = np.dot(W1, X) + b1
    A1 = np.maximum(Z1, 0) # ReLU function
    Z2 = np.dot(W2, A1) + b2
    A2 = Z2   # identity function on output layer
    
    # Compute and print loss
    loss = 0.5 * np.power(A2 - y, 2).sum()
    if (divmod(epoch, SKIP)[1]==0):
        print(epoch, loss)

    # Backpropagation to compute gradients of loss with respect to W1, W2, b1, and b2
    delta2 = (A2 - y)               # derivative of identity map == multiplying by ones
    grad_W2 = np.dot(delta2, A1.T)
    grad_b2 = np.dot(delta2, np.ones((N_batch, 1)))
    delta1 = np.dot(W2.T, delta2) * (Z1>0) # derivative of ReLU is a step function
    grad_W1 = np.dot(delta1, X.T)
    grad_b1 = np.dot(delta1, np.ones((N_batch, 1)))

    # Update weights & biases
    W1 = W1 - eta * grad_W1
    b1 = b1 - eta * grad_b1
    W2 = W2 - eta * grad_W2
    b2 = b2 - eta * grad_b2

0 10650668.534276865
100 3452.426188996744
200 837.9255822258376
300 468.6707231459839
400 359.30279055824855
500 312.5268766926531
600 288.68741151328794
700 274.35787230351264
800 264.59491718590203
900 257.23219442937324
1000 251.27483326674422
1100 246.23817944744587
1200 241.82254188466948
1300 237.85808002812507
1400 234.23716673669952
1500 230.88712818458936
1600 227.75967184124173
1700 224.84395216480584
1800 222.12253413207043
1900 219.5466404750544
2000 215.40689131116878
2100 212.92315011477206
2200 210.70063300293626
2300 208.5881065321572
2400 206.56508920483344


## 1. Convert the preceding code to use PyTorch Tensors instead of NumPy arrays

+ Replace use of `numpy.random.randn` with `torch.randn` to initialize the problem with PyTorch Tensors rather than NumPy arrays.
+ Replace instances of `np.dot` with [`torch.mm`](https://pytorch.org/docs/stable/torch.html#torch.mm) (both of which implement standard matrix-vector products).
+ Replace use of `np.ones` [`torch.ones`](https://pytorch.org/docs/stable/torch.html#torch.ones).
+ Replace the computation of `A1 = np.maximum(Z1, 0)` with a call to the PyTorch builtin function `torch.relu`.
+ Modify the computation of the `loss` to use PyTorch specific functions/methods (hint: there is a PyTorch `torch.Tensor.pow` method).
+ When printing the loss every hundred epochs, use the `.item()` method to extract its singleton scalar entry.
+ Make sure the loop executes in a similar fashion to the preceding loop.

In [28]:
N_batch, dimensions = 64, [784, 100, 10]

# Create random input and output data
X = torch.randn(dimensions[0], N_batch)
y = torch.randn(dimensions[-1], N_batch)

# Randomly initialize weights & biases
W1 = torch.randn(dimensions[1], dimensions[0])
W2 = torch.randn(dimensions[2], dimensions[1])
b1 = torch.randn(dimensions[1], 1)
b2 = torch.randn(dimensions[2], 1)

eta, MAXITER, SKIP = 5e-6, 2500, 100
for epoch in range(MAXITER):
    # Forward propagation: compute predicted y
    Z1 = torch.mm(W1, X) + b1
    A1 = torch.relu(Z1)        # Native PyTorch ReLU function
    Z2 = torch.mm(W2,A1) + b2
    A2 = Z2

    # Compute and print loss
    loss = 0.5*torch.Tensor.pow((A2-y),2).sum()
    if (divmod(epoch, SKIP)[1]==0):
        print(epoch, loss.item())

    # Backpropagation to compute gradients of loss with respect to W1, W2, b1, and b2
    delta2 = (A2 - y)               # derivative of identity map == multiplying by ones
    grad_W2 = torch.mm(delta2, A1.T)
    grad_b2 = torch.mm(delta2, torch.ones(N_batch, 1))
    delta1 = torch.mm(W2.T, delta2) * (Z1>0) # derivative of ReLU is a step function
    grad_W1 = torch.mm(delta1, X.T)
    grad_b1 = torch.mm(delta1, torch.ones(N_batch, 1))
    
    # Update weights & biases
    W1 = W1 - eta * grad_W1
    b1 = b1 - eta * grad_b1
    W2 = W2 - eta * grad_W2
    b2 = b2 - eta * grad_b2

0 12408148.0
100 3025.397216796875
200 745.8563232421875
300 449.88018798828125
400 373.4439392089844
500 342.5014953613281
600 325.57916259765625
700 313.908203125
800 304.8092346191406
900 297.2318420410156
1000 290.6766357421875
1100 284.8507080078125
1200 279.5747375488281
1300 274.78826904296875
1400 270.25927734375
1500 266.1265563964844
1600 262.2527160644531
1700 258.60675048828125
1800 255.16744995117188
1900 251.91770935058594
2000 248.84324645996094
2100 245.9315643310547
2200 243.17237854003906
2300 240.55593872070312
2400 238.0732879638672


## 2. Use `backward()` and `grad` to compute backpropagation and updates

Having set up the main loop with PyTorch Tensors, now make use of `autograd` to eliminate the tedious work of having to write the code to compute the gradients of the loss function with respect to `W1`, `W2`, `b1`, and `b2` explicitly.
+ Insert `requires_grad=True` as an argument in the construction of `W1`, `W2`, `b1`, and `b2`.
+ After computing the loss function value `loss`, replace all the lines used to compute gradients explicitly by a single call to `loss.backward()`.
+ Replace the update steps with gradients stored in `.grad` attributes of the weights & biases. For instance, you can now compute `W1 -= eta * W1.grad` *after* the call to `loss.backward()` rather than computing and explicitly storing `grad_W1` and later computing `W1 -= eta * grad_W1`.
    + Do these update steps within a `with torch.no_grad():` block (as provided below). The purpose of the [`torch.no_grad`](https://pytorch.org/docs/stable/torch.html#torch.no_grad) context manager is to reduce memory consumption.
    + After completing the updates, zero out the computed gradients before the next iteration by calling the method `.zero_()`. For instance, you would call `W1.grad.zero_()` to zero out the computed gradient in place. This call will be within the scope of the `torch.no_grad` context manager.

Notice, in PyTorch, methods like `.zero_` that have a training underscore in their name operate in place, i.e., they overwrite the memory locations associated with the tensor.

In [31]:
# Create random input and output data
X = torch.randn(dimensions[0], N_batch)
y = torch.randn(dimensions[-1], N_batch)

# Randomly initialize weights & biases
W1 = torch.randn(dimensions[1], dimensions[0], requires_grad=True)
W2 = torch.randn(dimensions[2], dimensions[1], requires_grad=True)
b1 = torch.randn(dimensions[1], 1, requires_grad=True)
b2 = torch.randn(dimensions[2], 1, requires_grad=True)

eta, MAXITER, SKIP = 5e-6, 2500, 100
for epoch in range(MAXITER):
    # Forward propagation: compute predicted y
    Z1 = torch.mm(W1, X) + b1
    A1 = torch.relu(Z1)        # Native PyTorch ReLU function
    Z2 = torch.mm(W2,A1) + b2
    A2 = Z2

    # Compute and print loss
    loss = 0.5*torch.Tensor.pow((A2-y),2).sum()
    if (divmod(epoch, SKIP)[1]==0):
        print(epoch, loss.item())

    # Backpropagation to compute gradients of loss with respect to W1, W2, b1, and b2
    loss.backward()

    # Update weights & biases
    with torch.no_grad():
        #
        # Fill in the code for the update steps
        W1 -= eta * W1.grad
        b1 -= eta * b1.grad
        W2 -= eta * W2.grad
        b2 -= eta * b2.grad
        # Manually zero the gradients after updating weights
        W1.grad.zero_()
        b1.grad.zero_()
        W2.grad.zero_()
        b2.grad.zero_()

0 10657143.0
100 604.24951171875
200 442.6575927734375
300 424.0107116699219
400 415.7649230957031
500 409.0162658691406
600 402.7860412597656
700 396.92584228515625
800 391.3941955566406
900 386.1683349609375
1000 379.3735046386719
1100 374.7746276855469
1200 370.4630432128906
1300 366.38568115234375
1400 362.5290222167969
1500 358.8803405761719
1600 355.42816162109375
1700 352.161376953125
1800 349.0693359375
1900 346.1419982910156
2000 343.3699035644531
2100 340.74420166015625
2200 338.2565002441406
2300 335.89886474609375
2400 333.66400146484375


---

# What next?

PyTorch has a large ecosystem of utilities including packages like  `torch.nn` (which is like Keras in spirit to simplify specifying a network architecture in an object-oriented way) and `torch.optim` (which makes managing different optimization schemes easier). We've covered a lot of ground in this tutorial so far, so this will be as far as we can get today. But you now should have enough of an understanding of backpropagation that you can pick up more at [`pytorch.org`](https://pytorch.org) independently.